In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

# orders
orders_df = pd.read_excel('order.xlsx', index_col=0)
orders_df['deadline'] = pd.to_datetime( orders_df['deadline'])

#product
product_df = pd.read_excel('product_2.xlsx', index_col=0)

#history
eqpt_history_df = pd.read_excel('eq_hist_data_2.xlsx')
eqpt_history_df['day'] = pd.to_datetime(eqpt_history_df['day'], unit='s')

#equipment
equipment_df = pd.read_excel('equipment_2.xlsx', index_col=0)
equipment_df['avg_estimated_speed_per_hour'] = \
    equipment_df['speed_per_hour'] * (1 - eqpt_history_df.groupby('id')['maintenance'].mean() / 24.0)

start_timestamp = np.datetime64('2019-03-18T00:00:00.000000000')

In [2]:
product_requirements = {}
for i in product_df.index:
    s = product_df['equipment_class'][i]
    processed_s = s.replace("'",'"').replace('\\xa0',' ')
    L = json.loads(processed_s)
    #product_requirements[i] = L
    prod_units = []
    for klas in L:
        prod_units.extend(equipment_df[equipment_df['class']==klas].index)
    product_requirements[i] = prod_units
#product_requirements

In [72]:
def create_empty_reservation_df():
    reservation_df = pd.DataFrame(columns=['equipment_id', 'order_id', 'amount', 'start', 'finish'])
    reservation_df['equipment_id'] = reservation_df['equipment_id'].astype(str)
    reservation_df['order_id'] = reservation_df['order_id'].astype(np.uint64)
    reservation_df['amount'] = reservation_df['amount'].astype(np.uint16)
    reservation_df['start'] = reservation_df['start'].astype(np.datetime64)
    reservation_df['finish'] = reservation_df['finish'].astype(np.datetime64)
    return reservation_df
def create_empty_load_df():
    load_df = pd.DataFrame(columns=['equipment_id', 'amount', 'avg_estimated_speed_per_hour'])
    load_df['equipment_id'] = load_df['equipment_id'].astype(np.float64)
    load_df['amount'] = load_df['amount'].astype(np.float64)
    load_df['avg_estimated_speed_per_hour'] = load_df['avg_estimated_speed_per_hour'].astype(np.float64)
    return load_df

In [96]:
def timedelta_to_hours(t_delta):
    return pd.to_timedelta(t_delta).total_seconds() / 3600
def get_last_stop_time(equipment_id, reservation_df):
    stops = reservation_df[reservation_df['equipment_id']==equipment_id]['finish']
    if ( stops.shape == (0,) ):
        return start_timestamp
    return reservation_df[reservation_df['equipment_id']==equipment_id]['finish'].max()

In [99]:
def calculate_loads(orders_df, reservation_df):
    # orders_df должен состоять из заказов с ОДИНАКОВЫМ дедлайном!!!
    deadline = orders_df.iloc[0]['deadline'] # TODO возможно поправить как-то даты? +1 день?
    #print(orders_df)
    #eq_loads = create_empty_load_df()
    eq_loads_d = {}
    n_orders = orders_df.shape[0]
    #print(n_orders)
    for i in range(n_orders):
        pill_id = orders_df.iloc[i]['product_id']
        pill_amount = orders_df.iloc[i]['amount']
        eq_units = product_requirements[pill_id]
        #print(eq_units)
        for eu in eq_units:
            #if ( eu not in eq_loads['equipment_id'] ):
            if ( eu not in eq_loads_d.keys() ):
                # available time
                aesph = equipment_df.loc[eu]['avg_estimated_speed_per_hour']
                freetime = timedelta_to_hours(deadline - get_last_stop_time(eu, reservation_df))
                eq_loads_d[eu] = {'potential':freetime * aesph,
                                  #'time_available': freetime,# TODO REMOVE
                                  'amount':pill_amount}
                #eq_loads_d[eu] = {'avg_estimated_speed_per_hour':equipment_df.loc[eu]['avg_estimated_speed_per_hour'],
                #                  'amount':pill_amount}
                
            else:
                #print(eu)
                #eq_loads.loc[eq_loads['equipment_id']==eu]['amount'] += pill_amount
                eq_loads_d[eu]['amount'] += pill_amount
    eq_loads = pd.DataFrame(eq_loads_d).T
    eq_loads['loadrate'] = eq_loads['amount'] / eq_loads['potential']
    return eq_loads

In [144]:
def place_order(order, reservation_df, eq_loads_df):
    pill_type = order['product_id']
    possible_eqpt = product_requirements[pill_type]
    possible_load_df = eq_loads_df.loc[possible_eqpt]
    eqpt_unit = possible_load_df[possible_load_df['loadrate']==possible_load_df['loadrate'].min()].iloc[0].name
    # .iloc[0] - если вдруг оборудования с минимальным loadrate больше одного
    start_time = get_last_stop_time(eqpt_unit, reservation_df)
    deadline = order['deadline']
    aesph = equipment_df.loc[eqpt_unit]['avg_estimated_speed_per_hour']
    freetime_hr = timedelta_to_hours(deadline - start_time)
    potential = freetime_hr * aesph
    if ( order['amount'] < potential ):
        predicted_work_time = np.ceil(order['amount'] / aesph * 3600 * 10**9) # наносекунды!
        finish_time = start_time + int(predicted_work_time) # выходные? ((((((
        reservation_df = reservation_df.append({'equipment_id': eqpt_unit,
                               'order_id': order.name,
                               'amount': order['amount'], 
                               'start': start_time,
                               'finish': finish_time},
                             ignore_index=True)
    else:
        can_do = int(np.floor(potential)) #сколько успеваем сделать
        reservation_df = reservation_df.append({'equipment_id': eqpt_unit,
                               'order_id': order.name,
                               'amount': can_do, 
                               'start': start_time,
                               'finish': deadline},
                             ignore_index=True)
        order_rest = order.copy()
        order_rest['amount'] = order['amount'] - can_do
        reservation_df = place_order(order_rest, reservation_df, eq_loads_df)
        # TOODOO eq_loads_df изменения!!!!!!
        
    return reservation_df

In [97]:
# не трогай пока
deadlines = orders_df['deadline'].unique()
reservation_df = create_empty_reservation_df()
deadlines.sort()
first_deadline = deadlines[0]
ldf = calculate_loads( orders_df[orders_df['deadline']==first_deadline], reservation_df )

In [115]:
eqpt_unit = ldf[ldf['loadrate']==ldf['loadrate'].min()].iloc[0].name
eqpt_unit
ldf.loc[eqpt_unit]
#pd.DataFrame.append

amount            52920.000000
potential          9491.481912
time_available       96.000000
loadrate              5.575526
Name: b1a710e7-7352-11e6-80f0-00505692e771, dtype: float64

In [127]:
order = orders_df[orders_df['deadline']==first_deadline].iloc[0]
order.copy

product_id    5c94953dc9e77c0001d5e142
amount                             680
deadline           2019-03-22 00:00:00
Name: 42, dtype: object

In [145]:
reservation_df = create_empty_reservation_df()
place_order(order, reservation_df, ldf)

,equipment_id,order_id,amount,start,finish
0,fd242b57-1c66-11e8-80e0-00155df02900,42.0,680,2019-03-18,2019-03-18 07:09:28.421052632


In [149]:
aesph = equipment_df.loc['fd242b57-1c66-11e8-80e0-00155df02900']['avg_estimated_speed_per_hour']
aesph
order['amount'] / aesph

7.157894736842105

In [150]:
0.157894736842105 * 60

9.4736842105263

In [151]:
.4736842105263 * 60

28.421052631578

In [125]:
ldf.loc[product_requirements['5c94953dc9e77c0001d5e141']]

,amount,potential,time_available,loadrate
2a3e1e73-8f22-11e7-80d8-00155df02900,121330.0,9600.000000,96.0,12.638542
055e6659-b576-11e7-80e0-00155df02900,236940.0,6720.000000,96.0,35.258929
4b002ba0-ea2c-11e3-8150-6ee664173b4c,244010.0,689.260078,96.0,354.017312


In [89]:
reservation_df = create_empty_reservation_df()
equipment_id='989'
stops = reservation_df[reservation_df['equipment_id']==equipment_id]['finish']
stops

Series([], Name: finish, dtype: datetime64[ns])

In [95]:
first_deadline

numpy.datetime64('2019-03-22T00:00:00.000000000')

In [98]:
ldf.sort_values(by='loadrate')

,amount,potential,time_available,loadrate
b1a710e7-7352-11e6-80f0-00505692e771,52920.0,9491.481912,96.0,5.575526
2dae72de-e3f8-11e7-80e0-00155df02900,57810.0,9600.000000,96.0,6.021875
5a0e4cb4-8f22-11e7-80d8-00155df02900,57810.0,9600.000000,96.0,6.021875
7da06b5d-a48c-11e5-80d6-00505692e771,58590.0,9600.000000,96.0,6.103125
5fda64fd-88db-11e3-8527-b614f2bbba79,58590.0,9600.000000,96.0,6.103125
0ea79b45-aa16-11e5-80d6-00505692e771,58690.0,9600.000000,96.0,6.113542
3c2ba553-8f22-11e7-80d8-00155df02900,56480.0,9120.000000,96.0,6.192982
1251252a-8f22-11e7-80d8-00155df02900,57810.0,9120.000000,96.0,6.338816
2a56384d-9f1c-11e5-80d6-00505692e771,57810.0,9120.000000,96.0,6.338816
8a1716d8-4f31-11e6-80e8-00505692e771,57810.0,9120.000000,96.0,6.338816


In [29]:
#ldf.loc[ldf['equipment_id']=='6fee9b9f-7a60-11e5-80d0-00505692e771']['amount'] += 205
ldf['loadrate'] = ldf['amount'] / ldf['avg_estimated_speed_per_hour']
ldf.sort_values(by='loadrate')
#ldf

,amount,avg_estimated_speed_per_hour,loadrate
b1a710e7-7352-11e6-80f0-00505692e771,52920.0,98.869603,535.250454
2dae72de-e3f8-11e7-80e0-00155df02900,57810.0,100.000000,578.100000
5a0e4cb4-8f22-11e7-80d8-00155df02900,57810.0,100.000000,578.100000
7da06b5d-a48c-11e5-80d6-00505692e771,58590.0,100.000000,585.900000
5fda64fd-88db-11e3-8527-b614f2bbba79,58590.0,100.000000,585.900000
0ea79b45-aa16-11e5-80d6-00505692e771,58690.0,100.000000,586.900000
3c2ba553-8f22-11e7-80d8-00155df02900,56480.0,95.000000,594.526316
1251252a-8f22-11e7-80d8-00155df02900,57810.0,95.000000,608.526316
2a56384d-9f1c-11e5-80d6-00505692e771,57810.0,95.000000,608.526316
8a1716d8-4f31-11e6-80e8-00505692e771,57810.0,95.000000,608.526316


In [ ]:
orders_df[orders_df['deadline']==first_deadline].shape()

In [ ]:
load_df = pd.DataFrame(columns=['equipment_id', 'order_id', 'amount', 'start', 'finish'])

load_df.info()

In [ ]:
eq_loads = create_empty_load_df()
'ec1133c9-2415-11e6-80e4-00505692e771' in eq_loads['equipment_id']

In [ ]:
orders_df.head()

In [ ]:
orders_df.sort_values(by=['deadline','amount'], ascending=[True,False])

In [ ]:
equipment_df.info()

In [ ]:
o = orders_df['deadline'].unique()
o.sort()
o[0]